### Notes on Linear Programming


For this, we need to install `PuLP`--a Python package for linear programming. 

In the `conda` environment, type:

```conda install -c conda-forge pulp```

For more info on `PuLP`, go to this [link](https://github.com/coin-or/pulp).

1. In using `PuLP` the key is to find the correct set of equations. 
2. Know if the problem is a maximization or minimization problem. 
3. Ensure that all constraints are encoded correctly. 

In [1]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpMinimize

#### The Spa Problem

- Maximize the profit: $ 350X1 +300X2  $
- Pump restrictions: $ 9X1 + 6X2 <= 1556 $
- Tubing restrictions: $ 12X1 + 16X2 <= 2880 $
- Non-zero restrictions: $X1, X2 >= 0 $


In [2]:
# Define the model
model = LpProblem(name="spa", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 3)}

# Add constraints
model += (x[1] + x[2] <= 200, "pumps")
model += (9 * x[1] + 6 * x[2] <= 1566, "labor")
model += (12 * x[1] + 16 * x[2] <= 2880, "tubing")
model += (x[1] >= 0, "positive-x1")
model += (x[2] >= 0, "positive-x2")


# Set the objective
model += 350 * x[1] + 300 * x[2]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")


status: 1, Optimal
objective: 66100.0
x1: 122.0
x2: 78.0
pumps: 0.0
labor: 0.0
tubing: -168.0
positive_x1: 122.0
positive_x2: 78.0


#### Unit Production Problem

Assign the following variables: 

- $X1$ Number of Model 1 to produce
- $X2$ Number of Model 2 to produce
- $X3$ Number of Model 3 to produce
- $X4$ Number of Model 1 to buy
- $X5$ Number of Model 2 to buy
- $X6$ Number of Model 3 to buy

Set the following constraints:

- $X1 + X4 = 3000$ Total number of Model 1 to produce
- $X2 + X5 = 2000$ Total number of Model 2 to produce
- $X3 + X6 = 900$ Total number of Model 3 to produce
- $ 2X1 + 1.5X2 + 2X3 <= 10000 $ The wiring limit
- $ 1X1 + 2X2 + 1X3 <= 5000 $ The harness limit

Then maximize this equation (the coefficients are the price to |produce/buy): 

$ 50 * X1 + 83 * X2 + 130 * X3 + 61 * X4 + 97 * X5 + 145 * X6 $ 

In [3]:
# Define the model
model = LpProblem(name="unit", sense=LpMinimize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 7)}

# Add constraints
model += (2 * x[1] + 1.5 * x[2] + 3 * x[3] <= 10000, "wiring")
model += (1 * x[1] + 2 * x[2] + 1 * x[3] <= 5000, "harnessing")
model += (x[1] +  x[4] == 3000, "model1")
model += (x[2] +  x[5] == 2000, "model2")
model += (x[3] +  x[6] == 900, "model3")


# Set the objective
model += 50 * x[1] + 83 * x[2] + 130 * x[3] + 61 * x[4] + 97 * x[5] + 145 * x[6]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")


status: 1, Optimal
objective: 453300.0
x1: 3000.0
x2: 550.0
x3: 900.0
x4: 0.0
x5: 1450.0
x6: 0.0
wiring: -475.0
harnessing: 0.0
model1: 0.0
model2: 0.0
model3: 0.0


#### The Investment Problem

- Total money to invest: $750,000$

Assign the following variables: 

    - X1 Amount to invest in AcmeChemical
    - X2 Amount to invest in DynaStar
    - X3 Amount to invest in Eagle Vision
    - X4 Amount to invest in Micr0 Modelling
    - X5 Amount to invest in OptiPro
    - X6 Amount to invest in Sabre Systems

- No amount should be greater than $ 187,500 $
- $X1$, $X2$, and $X4$ combined should form at least half of the total investment.
- $X2$, $X3$, and $X5$ combined must not exceed $ 262,500 $
- All values must be positive.

In [4]:
# Define the model
model = LpProblem(name="invest", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 7)}

# Add constraints
model += (x[1] + x[2] + x[3] + x[4] + x[5] + x[6] == 750000, "total")
for i in range(1,7):
    model += (x[i] <= 187500, f"quarter_restriction{i}")
model += (x[1] + x[2] + x[4] + x[6] >= 375000, "restriction7")
model += (x[2] + x[3] + x[5] <= 262500, "restriction8")
for i in range(1,7):
    model += (x[i] >=0, f"positive_restriction{i}")

# Set the objective
model += 0.0865 * x[1] + 0.095 * x[2] + 0.1 * x[3] + 0.0875 * x[4] + 0.0925 * x[5] + 0.09 * x[6]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 68887.5
x1: 112500.0
x2: 75000.0
x3: 187500.0
x4: 187500.0
x5: 0.0
x6: 187500.0
total: 0.0
quarter_restriction1: -75000.0
quarter_restriction2: -112500.0
quarter_restriction3: 0.0
quarter_restriction4: 0.0
quarter_restriction5: -187500.0
quarter_restriction6: 0.0
restriction7: 187500.0
restriction8: 0.0
positive_restriction1: 112500.0
positive_restriction2: 75000.0
positive_restriction3: 187500.0
positive_restriction4: 187500.0
positive_restriction5: 0.0
positive_restriction6: 187500.0


#### The Toy Problem

- $X1$ Number of planes to produce
- $X2$ Number of boats to product
- The making limit: $ 3X1 + X2 <= 120 $
- The finishing limit: $X1 + 2X2 <= 160 $
- Boat limit:  $x<= 35$
- Non-zero: $X1, X2 >= 0 $

In [5]:
# Define the model
model = LpProblem(name="pototoy", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 3)}

# Add constraints
model += (3 * x[1] + x[2]  <= 120, "making")
model += (x[1] + 2 * x[2]  <= 160, "finishing")
model += (x[1]  <= 35, "planes")
model += (x[1] >= 0, "non-zero1")
model += (x[2] >= 0, "non-zero2")

# Set the objective
model += 7 * x[1] + 6 * x[2] 

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 544.0
x1: 16.0
x2: 72.0
making: 0.0
finishing: 0.0
planes: -19.0
non_zero1: 16.0
non_zero2: 72.0


#### The Mamaro

Let $X1$ => Insurance, and $X2$ => Mortgage

- Underwriting: $3X1 + 2X2 <= 2400$
- Admin: $X2 <= 800$
- Claims: $2X1 <= 1200$
- Maximize: $5X1 + 2X2$

In [6]:
# Define the model
model = LpProblem(name="contest", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 3)}

# Add constraints
model += (3 * x[1] + 2 * x[2]  <= 2400, "underwriting")
model += ( x[2]  <= 800, "admin")
model += (2 * x[1]  <= 1200, "claims")
model += (x[1] >= 0, "non-zero1")
model += (x[2] >= 0, "non-zero2")

# Set the objective
model += 5 * x[1] + 2 * x[2] 

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 3600.0
x1: 600.0
x2: 300.0
underwriting: 0.0
admin: -500.0
claims: 0.0
non_zero1: 600.0
non_zero2: 300.0


#### Corn and Soy Meal Problem

Let $X1$ and $X2$ for the amount of corn and soy meal in lbs, respectively. 

Constraints:

- $(1/16) * X1 + (1.5/16) * X2 >= 45/16 $ on N1 content
- $(2/16) * X1 + (0.5/16) * X2 >= 40/16 $ on N2 content
- $ X1 + X2 <= 40 $ on the total 
- Non-zero : $X1, X2 >= 0 $

In [10]:
# Define the model
model = LpProblem(name="contest2", sense=LpMinimize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 3)}

# Add constraints
model += ( (1/16) * x[1] + (1.5/16) * x[2] >= 45/16, "n1")
model += ( (2/16) * x[1] + (0.5/16) * x[2] >= 40/16, "n2")
model += (x[1] + x[2]  <= 40, "total")
model += (x[1] >= 0, "non-zero1")
model += (x[2] >= 0, "non-zero2")

# Set the objective
model += 40 * x[1] + 80 * x[2] 

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 2000.0
x1: 30.0
x2: 10.0
n1: 0.0
n2: 1.5625
total: 0.0
non_zero1: 30.0
non_zero2: 10.0


In [14]:
# Define the model
model = LpProblem(name="question2", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 3)}

# Add constraints
model += (6 * x[1] + 5 * x[2] <= 60, "restriction1")
model += (2 * x[1] + 3 * x[2] <= 24, "restriction2")
model += (3 * x[1] + 6 * x[2] <= 48, "restriction3")
model += (x[1] >= 0, "non-zero1")
model += (x[2] >= 0, "non-zero2")

# Set the objective
model += 2 * x[1] + 5 * x[2] 

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 40.0
x1: 0.0
x2: 8.0
restriction1: -20.0
restriction2: 0.0
restriction3: 0.0
non_zero1: 0.0
non_zero2: 8.0


#### Blacktop Refining

Let: 

$ X_{1} $, be Type 1
$ X_{2} $, be Type 2 

Systems of equation:

For Cu:
$$ 0.2X_{1} + 0.3X_{2} >= 8 $$ 

For Zn:
$$ 0.2X_{1} + 0.25X_{2} >= 6 $$ 

For Mg:
$$ 0.15X_{1} + 0.1X_{2} >= 5 $$ 

Optimal Solution:

$$ 90X_{1} + 120X_{2} $$

In [12]:
# Define the model
model = LpProblem(name="question2", sense=LpMinimize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 3)}

# Add constraints
model += (0.2 * x[1] + 0.3 * x[2] >= 8, "restriction1")
model += (0.2 * x[1] + 0.25 * x[2] >= 6, "restriction2")
model += (0.15 * x[1] + 0.1 * x[2] >= 5, "restriction3")
model += (x[1] >= 0, "non-zero1")
model += (x[2] >= 0, "non-zero2")

# Set the objective
model += 90 * x[1] + 120 * x[2] 

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 3480.0
x1: 28.0
x2: 8.0
restriction1: 4.440892098500626e-16
restriction2: 1.6000000000000005
restriction3: 2.220446049250313e-16
non_zero1: 28.0
non_zero2: 8.0


In [13]:
# Define the model
model = LpProblem(name="question2", sense=LpMinimize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 3)}

# Add constraints
model += (.2* x[1] + .3 * x[2] >= 8, "restriction1")
model += (.2 * x[1] + .25 * x[2] >= 6, "restriction2")
model += (.15* x[1] + (1/10) * x[2] >= 5, "restriction3")
model += (x[1] >= 0, "non-zero1")
model += (x[2] >= 0, "non-zero2")

# Set the objective
model += 90 * x[1] + 120 * x[2] 

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 3480.0
x1: 28.0
x2: 8.0
restriction1: 4.440892098500626e-16
restriction2: 1.6000000000000005
restriction3: 2.220446049250313e-16
non_zero1: 28.0
non_zero2: 8.0
